In [1]:
import numpy as np
np.random.seed(42)

In [2]:
import pandas as pd
df = pd.read_parquet("user_item_ratio_stacked.parquet")
df.columns = ["userID", "itemID", "rating"]

In [3]:
filtered_df = df.groupby('userID')['itemID'].count().reset_index().loc[lambda x: x['itemID'] > 2]


In [4]:
df= df.loc[df.userID.isin(filtered_df.userID)]

In [5]:
df.columns = ['user', 'item', 'rating']

In [13]:
%time
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable


# Getting the number of users and items (movies) from the original data
nb_users = df['user'].nunique()
nb_items = df['item'].nunique()

# Converting the data into Torch tensors
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_items = data['item'][data['user'] == id_users]
        id_ratings = data['rating'][data['user'] == id_users]
        ratings = np.zeros(nb_items)
        ratings[id_items - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

# Convert the data to the required format
training_set = convert(df)

# Convert to PyTorch FloatTensor
training_set = torch.FloatTensor(training_set)

# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_items, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_items)
        self.activation = nn.Sigmoid()
    
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

nb_epoch = 100
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.requires_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_items / float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data.item() * mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s))

test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.requires_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_items / float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data.item() * mean_corrector)
        s += 1.
print('test loss: ' + str(test_loss / s))


CPU times: user 15 µs, sys: 10 µs, total: 25 µs
Wall time: 217 µs
epoch: 1 loss: 0.3220489247997173
epoch: 2 loss: 0.24130308307503875
epoch: 3 loss: 0.23841300543556943
epoch: 4 loss: 0.23637676441896477
epoch: 5 loss: 0.23489531637803368
epoch: 6 loss: 0.23383774212907232
epoch: 7 loss: 0.23256486499101714
epoch: 8 loss: 0.2316500693050904
epoch: 9 loss: 0.2304656144733386
epoch: 10 loss: 0.22959655371237928
epoch: 11 loss: 0.22849395375470044
epoch: 12 loss: 0.22754212227155285
epoch: 13 loss: 0.2265186784114771
epoch: 14 loss: 0.22556802612158014
epoch: 15 loss: 0.22456496383725202
epoch: 16 loss: 0.22357941760242972
epoch: 17 loss: 0.22258790890090327
epoch: 18 loss: 0.22162400579158056
epoch: 19 loss: 0.2206351567419211
epoch: 20 loss: 0.21972368932521325
epoch: 21 loss: 0.21881596365040626
epoch: 22 loss: 0.21790689773873748
epoch: 23 loss: 0.21702493029631548
epoch: 24 loss: 0.21619407401378332
epoch: 25 loss: 0.21534831281192918
epoch: 26 loss: 0.2145110988280184
epoch: 27 los

In [11]:
df

,user,item,rating
27,15913.0,2,0.352941
28,15913.0,7,0.088083
29,15913.0,12,0.957760
38,15921.0,2,0.647059
39,15921.0,4,0.752212
...,...,...,...
1090476,1542559.0,2,0.428571
1090477,1542559.0,6,0.567568
1090492,1542573.0,0,0.333333
1090493,1542573.0,2,0.428571


In [24]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

# Load your DataFrame with columns ['user', 'item', 'rating']
# Modify the file paths accordingly
# Example:
# df = pd.read_csv('your_data_file.csv')

# Getting the number of users and movies
nb_users = int(df['user'].nunique())
nb_movies = int(df['item'].nunique())

# Create a user-item matrix (ratings matrix)
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data['item'][data['user'] == id_users]
        id_ratings = data['rating'][data['user'] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies.values - 1] = id_ratings.values
        new_data.append(list(ratings))
    return new_data

# Convert the DataFrame to the user-item matrix
data = convert(df)

# Convert the data into Torch tensors
data = torch.FloatTensor(data)

# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

nb_epoch = 256
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(data[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.requires_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s))

# Now, for evaluation, you can use the following code:
# Assuming you have a separate evaluation DataFrame called 'eval_df' with columns ['user', 'item', 'rating']

user_indices = eval_df['user'].values - 1
item_indices = eval_df['item'].values - 1
predicted_ratings = sae(data).detach().numpy()
predicted_ratings = predicted_ratings[user_indices, item_indices]

eval_df['Predicted'] = predicted_ratings
print(eval_df)


epoch: 1 loss: tensor(0.3252)
epoch: 2 loss: tensor(0.2413)
epoch: 3 loss: tensor(0.2383)
epoch: 4 loss: tensor(0.2362)
epoch: 5 loss: tensor(0.2347)
epoch: 6 loss: tensor(0.2337)
epoch: 7 loss: tensor(0.2325)
epoch: 8 loss: tensor(0.2317)
epoch: 9 loss: tensor(0.2305)
epoch: 10 loss: tensor(0.2297)
epoch: 11 loss: tensor(0.2286)
epoch: 12 loss: tensor(0.2277)
epoch: 13 loss: tensor(0.2267)
epoch: 14 loss: tensor(0.2257)
epoch: 15 loss: tensor(0.2247)
epoch: 16 loss: tensor(0.2237)
epoch: 17 loss: tensor(0.2228)
epoch: 18 loss: tensor(0.2219)
epoch: 19 loss: tensor(0.2210)
epoch: 20 loss: tensor(0.2200)
epoch: 21 loss: tensor(0.2190)
epoch: 22 loss: tensor(0.2181)
epoch: 23 loss: tensor(0.2172)
epoch: 24 loss: tensor(0.2164)
epoch: 25 loss: tensor(0.2156)
epoch: 26 loss: tensor(0.2148)
epoch: 27 loss: tensor(0.2139)
epoch: 28 loss: tensor(0.2129)
epoch: 29 loss: tensor(0.2122)
epoch: 30 loss: tensor(0.2120)
epoch: 31 loss: tensor(0.2110)
epoch: 32 loss: tensor(0.2098)
epoch: 33 loss: t

IndexError: arrays used as indices must be of integer (or boolean) type

In [17]:
# Tính dự đoán trên tập kiểm tra bằng mô hình SAE
eval_loader = DataLoader(eval_set, batch_size=1)
predictions = []
for data in eval_loader:
    input_data = Variable(data)
    output = sae(input_data)
    predictions.append(output.data.numpy())


In [25]:
import pandas as pd

# Chuyển đổi danh sách dự đoán thành mảng NumPy
predictions = np.concatenate(predictions, axis=0)

# Tạo DataFrame mới
result_df = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': eval_set.numpy().flatten()})


In [28]:
result_df.Predicted.max()

0.5013319253921509

In [29]:
result_df.Predicted.min()

0.49811220169067383